In [27]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from linearmodels import PanelOLS
import numpy as np
import patsy
from numpy.polynomial.polynomial import polyfit

# 1. Read the data
df = pd.read_csv("GMdata.csv",sep='\t')

# 2. Basic summary
print("\nDescriptive statistics:")
print(df.describe())

# 3. Check the time dimension per firm
#    Count how many years each firm has
year_counts = df.groupby('index')['yr'].nunique()

# 4. Identify the balanced panel
#    Suppose we expect 4 unique years (73, 78, 83, 88) for each firm
balanced_firms = year_counts[year_counts == 4].index

# Create separate dataframes
df_balanced = df[df['index'].isin(balanced_firms)].copy()
df_unbalanced = df.copy()  # Entire dataset

print("\nNumber of firms in balanced panel:", len(balanced_firms))
print("Number of firms (total) in unbalanced panel:", df['index'].nunique())

# 5. Summarize balanced and unbalanced panels
print("\nBalanced panel summary:")
print(df_balanced.describe())

print("\nUnbalanced panel summary:")
print(df_unbalanced.describe())



Descriptive statistics:
             index         sic3           yr        ldsal         lemp  \
count  2971.000000  2971.000000  2971.000000  2971.000000  2971.000000   
mean    696.203299   331.455739    80.489061     5.673087     1.259177   
std     404.779371    51.952189     5.351874     1.960717     1.775248   
min       1.000000   200.000000    73.000000    -0.857349    -3.772261   
25%     343.500000   286.000000    78.000000     4.250526    -0.024805   
50%     696.000000   356.000000    78.000000     5.529348     1.114157   
75%    1048.000000   367.000000    83.000000     7.083825     2.631889   
max    1400.000000   399.000000    88.000000    11.698400     6.732211   

             ldnpt        ldrst        ldrnd        ldinv  
count  2971.000000  2971.000000  2971.000000  2971.000000  
mean      4.468996     3.400962     1.787530     2.674828  
std       2.216520     2.028775     2.052410     2.170476  
min      -1.389284    -4.287164    -5.313206    -3.844328  
25%     

In [ ]:


# Balanced panel regression (no dummies)
model_bal_basic = smf.ols("ldsal ~ lemp + ldpt",
                          data=df_balanced).fit()
print("OLS (Balanced), no dummies:")
print(model_bal_basic.summary())

ModuleNotFoundError: No module named 'statsmodels'

In [3]:
# Unbalanced panel regression (no dummies)
model_unbal_basic = smf.ols("ldsal ~ lemp + ldpt",
                            data=df_unbalanced).fit()
print("\nOLS (Unbalanced), no dummies:")
print(model_unbal_basic.summary())

NameError: name 'smf' is not defined

In [9]:
# Example with unbalanced panel and dummies:
model_unbal_dummies = smf.ols("ldsal ~ lemp + ldpt + C(yr) + C(sic3)",
                              data=df_unbalanced).fit()
print("\nOLS (Unbalanced), with year & industry dummies:")
print(model_unbal_dummies.summary())

NameError: name 'smf' is not defined

In [ ]:

# First, we need a multi-index: (firm, year)
df_panel = df_unbalanced.set_index(['index','yr']).sort_index()

# The PanelOLS formula approach:
# Here 'ldsal' is the dependent variable,
# 'lemp' and 'ldpt' are regressors, plus we include "EntityEffects" and "TimeEffects".
fe_model = PanelOLS.from_formula(
    formula='ldsal ~ 1 + lemp + ldpt + EntityEffects + TimeEffects',
    data=df_panel
).fit()
print(fe_model)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 52.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


NameError: name 'df_unbalanced' is not defined

In [ ]:


# Let's pretend we have 'survive' in df_unbalanced.
# 1) Build design matrix with dummies for year and industry:
df_unbalanced['intercept'] = 1
X_cols = ['intercept', 'ldinv', 'ldpt']

# Add year and industry dummies (one approach is to get dummies and then drop one)
df_dummies = pd.get_dummies(df_unbalanced[['yr','sic3']], prefix=['yr','sic3'], drop_first=True)
X = pd.concat([df_unbalanced[X_cols], df_dummies], axis=1)
y = df_unbalanced['survive']  # must exist in your data

# 2) Fit probit model
probit_model = sm.Probit(y, X, missing='drop').fit()
print(probit_model.summary())

# Interpret signs of coefficients on ldinv, ldpt, etc.

NameError: name 'df_unbalanced' is not defined

In [ ]:


# Assume we have: y = ldsal, labor = lemp, capital = ldpt, investment = ldinv.
# 1) First stage: y - beta_l * l = phi(k, i) + eps
#    Where phi(...) is approximated by a polynomial or spline in k and i.

# We guess an initial labor coefficient from OLS or from prior knowledge:
initial_beta_l = 0.5  # example placeholder

# Construct the dependent variable for the first stage
df_unbalanced['y_tilde'] = df_unbalanced['ldsal'] - initial_beta_l * df_unbalanced['lemp']

# Let's do a simple polynomial in capital and investment:
df_unbalanced['k'] = df_unbalanced['ldpt']
df_unbalanced['i'] = df_unbalanced['ldinv']

# Example: a second-order polynomial in (k, i)
#  y_tilde = a0 + a1*k + a2*i + a3*k^2 + a4*i^2 + a5*k*i + error
first_stage_model = smf.ols("y_tilde ~ k + i + I(k**2) + I(i**2) + I(k*i)",
                            data=df_unbalanced).fit()
print(first_stage_model.summary())

# Predicted "phi_hat"
df_unbalanced['phi_hat'] = first_stage_model.fittedvalues

# 2) Second stage: identify the capital coefficient by projecting out the expected productivity
#    and controlling for survival or selection. In OP, we use the "phi_hat" and "phi_hat_lag"
#    plus survival corrections. The details can be quite involved.

# For illustration:
df_unbalanced['phi_hat_lag'] = df_unbalanced.groupby('index')['phi_hat'].shift(1)

# Then we might run:
second_stage_model = smf.ols("ldsal ~ lemp + ldpt + phi_hat_lag",
                             data=df_unbalanced.dropna()).fit()
print(second_stage_model.summary())

# The coefficient on ldpt would be your OP estimate of capital's coefficient.
# If you want to incorporate selection controls, you'd add an inverse Mills ratio
# from a survival probit, or a polynomial in phi_hat_lag. This is just a skeleton.

NameError: name 'df_unbalanced' is not defined

In [8]:
# 1) Suppose second_stage_model from OP gave us fitted values or residuals as firm TFP
df_unbalanced['tfp_hat'] = second_stage_model.resid  # or fitted productivity measure

# 2) Compute firm-year market share within each SIC3:
df_unbalanced['sector_sales'] = df_unbalanced.groupby(['sic3','yr'])['ldsal'].transform(lambda x: x.sum())
# CAREFUL: 'ldsal' is log(sales). We might want actual sales, not logs.
# If you only have logs, do something like:
df_unbalanced['sales'] = df_unbalanced['ldsal'].apply(np.exp)  # approximate real sales
df_unbalanced['sector_sales'] = df_unbalanced.groupby(['sic3','yr'])['sales'].transform('sum')
df_unbalanced['mshare'] = df_unbalanced['sales'] / df_unbalanced['sector_sales']

# 3) Weighted TFP at sector-year level
df_unbalanced['weighted_tfp'] = df_unbalanced['mshare'] * df_unbalanced['tfp_hat']

# 4) Aggregate by sector-year
sector_year_tfp = df_unbalanced.groupby(['sic3','yr'])['weighted_tfp'].sum().reset_index()
sector_year_tfp.rename(columns={'weighted_tfp':'aggregate_tfp'}, inplace=True)

print(sector_year_tfp)

NameError: name 'second_stage_model' is not defined